# Basic Practice

In [1]:
pip install -q langchain-core requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
import requests

In [3]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    return a * b

In [4]:
print(multiply.invoke({'a':10, 'b':20}))

200


In [5]:
multiply.name

'multiply'

In [6]:
multiply.description

'Multiplies two numbers.'

In [7]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [45]:
llm = ChatGroq(model="llama3-70b-8192")

In [10]:
llm.invoke("hi")

AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_time': 0.071428571, 'prompt_time': 0.000128159, 'queue_time': 0.052390888, 'total_time': 0.07155673}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run-ef35f416-3794-4283-8db6-fe64a2f0ebeb-0', usage_metadata={'input_tokens': 11, 'output_tokens': 25, 'total_tokens': 36})

In [11]:
llm_with_tool = llm.bind_tools([multiply])

In [12]:
llm_with_tool.invoke("hi")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_x3tw', 'function': {'arguments': '{"a":1,"b":1}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 916, 'total_tokens': 972, 'completion_time': 0.16, 'prompt_time': 0.038490166, 'queue_time': 0.059089144, 'total_time': 0.198490166}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-35826f9b-756b-4704-a4d3-cadbb1868780-0', tool_calls=[{'name': 'multiply', 'args': {'a': 1, 'b': 1}, 'id': 'call_x3tw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 916, 'output_tokens': 56, 'total_tokens': 972})

In [13]:
query = HumanMessage('can you multiply 10 and 20?')

In [14]:
message = [query]

In [15]:
message

[HumanMessage(content='can you multiply 10 and 20?', additional_kwargs={}, response_metadata={})]

In [16]:
result= llm_with_tool.invoke(message)

In [17]:
message.append(result)

In [18]:
message

[HumanMessage(content='can you multiply 10 and 20?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2ckj', 'function': {'arguments': '{"a":10,"b":20}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 924, 'total_tokens': 972, 'completion_time': 0.137142857, 'prompt_time': 0.0300102, 'queue_time': 0.055213609999999996, 'total_time': 0.167153057}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d61a2984-7267-457d-8d00-4c34434987db-0', tool_calls=[{'name': 'multiply', 'args': {'a': 10, 'b': 20}, 'id': 'call_2ckj', 'type': 'tool_call'}], usage_metadata={'input_tokens': 924, 'output_tokens': 48, 'total_tokens': 972})]

In [19]:
tool_result = multiply.invoke(result.tool_calls[0])

In [20]:
tool_result

ToolMessage(content='200', name='multiply', tool_call_id='call_2ckj')

In [21]:
message.append(tool_result)

In [22]:
llm_with_tool.invoke(message).content

'The answer is 200.'

# Currency Conversion Tool

In [ ]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """Get conversion factor from base currency to target currency."""
    url = f"https://v6.exchangerate-api.com/v6/7{api_key}/pair/{base_currency}/{target_currency}"
    response = requests.get(url)
    return response.json()

@tool
def convert(base_currency_value: float, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """Convert base currency value to target currency value."""
    return base_currency_value * conversion_rate

In [47]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'number'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [48]:
get_conversion_factor.invoke({'base_currency': 'USD', 'target_currency': 'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1745798401,
 'time_last_update_utc': 'Mon, 28 Apr 2025 00:00:01 +0000',
 'time_next_update_unix': 1745884801,
 'time_next_update_utc': 'Tue, 29 Apr 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4741}

In [49]:
convert.invoke({'base_currency_value': 100, 'conversion_rate': 85.4741})

8547.41

In [50]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [51]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [52]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [53]:
ai_message = llm_with_tools.invoke(messages)

In [54]:
messages.append(ai_message)

In [55]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_yre0',
  'type': 'tool_call'}]

In [56]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [57]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yre0', 'function': {'arguments': '{"base_currency":"INR","target_currency":"USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 1053, 'total_tokens': 1106, 'completion_time': 0.152903222, 'prompt_time': 0.033527228, 'queue_time': 0.056341146, 'total_time': 0.18643045}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8bcb48e1-ec20-452a-a1cc-173993e28574-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'call_yre0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1053, 'output_tokens': 53, 'total_tokens': 1106}),

In [58]:
llm_with_tools.invoke(messages).content

''

In [59]:
from langchain.agents import initialize_agent, AgentType


agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True 
)

C:\Users\jadit\AppData\Local\Temp\ipykernel_20544\1361761902.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [60]:
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
I'm doing well, thanks! I'm here to help you with any questions or tasks you might have.

Action:
```
{
  "action": "No action needed",
  "action_input": "Just a friendly greeting"
}
```

Observation: No action needed is not a valid tool, try one of [get_conversion_factor, convert].
Thought:I apologize for the mistake!

Action:
```
{
  "action": "Final Answer",
  "action_input": "I'm doing well, thanks! I'm here to help you with any questions or tasks you might have."
}
```



> Finished chain.
